In [1]:
# import libraries
# lib for pre trained model 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
# lib for layers
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
# lib for connecting pretrained and created model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
#lib for compile
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

# lib for pre processing
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
# lib for spliting i/p and o/p
from sklearn.model_selection import train_test_split
# lib for metrics
from sklearn.metrics import classification_report
# other lib
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [2]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

DIRECTORY = r"C:\HOPE\Deep Learning\Helmet detection\dataset"
CATEGORIES = ["with_Helmet", "without_Helmet"]

In [8]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)

        data.append(image)
        labels.append(category)

[INFO] loading images...


In [9]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
      test_size=0.20, stratify=labels, random_state=42)

In [10]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [11]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
       input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

C:\Users\kathi\AppData\Local\Temp\ipykernel_5416\633592287.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  baseModel = MobileNetV2(weights="imagenet", include_top=False,


In [12]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False


print("[INFO] Compiling model....")
# initialize the initial learning rate, number of epochs to train for,
# and batch size

opt = SGD(learning_rate=INIT_LR)
model.compile(loss="binary_crossentropy",optimizer =opt,
              metrics =["accuracy"])

[INFO] Compiling model....


In [13]:
# train the head of the network
# calculating steps per epoch and validation steps
steps_per_epoch = len(trainX)//BS
validation_steps = len(testX)//BS

# Use .repeat() to ensure the dataset generates enough batches
train_data_gen = aug.flow(trainX, trainY, batch_size=BS)
validation_data_gen = ImageDataGenerator().flow(testX, testY, batch_size=BS)

#train the model

print("[INFO] training ahead....")
H = model.fit(train_data_gen,
              steps_per_epoch=steps_per_epoch,
              validation_data = validation_data_gen,
              validation_steps=validation_steps,
              epochs =EPOCHS)


[INFO] training ahead....
Epoch 1/20


C:\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 51s 752ms/step - accuracy: 0.3212 - loss: 1.1317 - val_accuracy: 0.2792 - val_loss: 0.9756
Epoch 2/20
 1/61 ━━━━━━━━━━━━━━━━━━━━ 28s 479ms/step - accuracy: 0.3750 - loss: 1.0123

C:\Anaconda\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3750 - loss: 1.0123 - val_accuracy: 0.2857 - val_loss: 1.0343
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 46s 733ms/step - accuracy: 0.3968 - loss: 0.9664 - val_accuracy: 0.3688 - val_loss: 0.8401
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5312 - loss: 0.8208 - val_accuracy: 0.2857 - val_loss: 0.8568
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 45s 714ms/step - accuracy: 0.5222 - loss: 0.8240 - val_accuracy: 0.5021 - val_loss: 0.7371
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5938 - loss: 0.7880 - val_accuracy: 0.4286 - val_loss: 0.7836
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 44s 695ms/step - accuracy: 0.6236 - loss: 0.7316 - val_accuracy: 0.6292 - val_loss: 0.6627
Epoch 8/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5938 - loss: 0.7609 - val_accuracy: 0.7143 - val_loss: 0.7028
Epoch 9/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 44s 694ms/step - accuracy: 0.6979 - loss: 0.6626 - val_accuracy: 0.7292 - val_loss

In [14]:

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

[INFO] evaluating network...
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 494ms/step


In [15]:
# classification report
print(classification_report(testY.argmax(axis =1), predIdxs, target_names=lb.classes_))

                precision    recall  f1-score   support

   with_Helmet       0.95      0.36      0.52       108
without_Helmet       0.85      0.99      0.92       386

      accuracy                           0.86       494
     macro avg       0.90      0.68      0.72       494
  weighted avg       0.87      0.86      0.83       494



In [16]:
#Save the model
print("[INFO] saving the helmet detector model...")
model.save('helmet_detector.h5')

[INFO] saving the helmet detector model...
